# RUN IN JUPYTER
このノートブックはニューラルネットワークの準同期式更新アルゴリズムを初めて使う人のためのチュートリアルです

このチュートリアルを通して、準同期式の再学習を体験してみましょう

## 1. 必要なライブラリをインストールしよう

このモデルはPyTorchで実装されています

In [1]:
# データセット
from procedure.preprocess import cifar_10_for_vgg_loaders as cifar10
# フレームワーク
import random
import numpy as np
import torch
# 準同期式レイヤ
from layers.static import OptimizedSemiSyncLinear
# モデル
from torchvision.models import vgg
# 評価、更新用関数
import torch.nn as nn
import torch.optim as optim
# 時間
import datetime
# 統計的な評価
from sklearn.metrics import classification_report

## 2. 乱数シードを固定しよう

プログラム内で実行するたびに違う数字を発生させる場合があります。
そうなった場合、出力された結果がアルゴリズムの差によるものなのか、それとも乱数によるものなのかがわからなくなります。
それを防ぐために、乱数シードを固定しておきましょう

In [2]:
torch.manual_seed(1234)
np.random.seed(1234)
random.seed(1234)

## 3. 学習データを読み込もう

このノートブックではcifer10というデータセットを利用します

In [3]:
train_loader, test_loader = cifar10()

Files already downloaded and verified
Files already downloaded and verified


## 4. モデルを読み込もう

このノートブックではvgg16というモデルを利用します

In [4]:
model = vgg.vgg16()

## 5. 準同期式に変更しよう

vgg16の全結合層を順同期式に変更しましょう

In [5]:
model.classifier[0] = OptimizedSemiSyncLinear(model.classifier[0])
model.classifier[3] = OptimizedSemiSyncLinear(model.classifier[3])

## 6. モデルを学習モードに変更しよう

PyTorchのモデルには評価モード(eval)と学習モード(train)があります
今回は再学習をさせるので、学習モードにしておきましょう

In [6]:
model = model.train()

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

## 7. 評価関数と最適化関数を定義しよう

逆伝播をする前に、順伝播で得られた結果を**評価**します。
そして、評価値(loss)をもとに逆伝播を行い、各パラメータを**最適化**していきます。

このときの評価と最適化に使う関数を定義しておきましょう

In [7]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

## 8. GPUを使ってみよう

この学習を行う際にはたくさんの計算が行われます。CPUだけでは時間がかるのでGPUを使ってみましょう

In [8]:
device = torch.device('cuda')
model = model.to(device)

## 9. 学習の流れを定義しよう

学習を１回するたびに以下のプログラムを動かす必要があります。

1. データの読み込み
2. 勾配の初期化
3. 順伝播
4. 評価
5. 逆伝播
6. パラメータ更新

これらのプログラムは何回も実行するものなので、１つの関数として定義しておきましょう

In [9]:
def train(epoch):
    total_loss = 0
    total_size = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        # データセットをGPUに対応
        data, target = data.to(device), target.to(device)
        # 勾配を初期化
        optimizer.zero_grad()
        # forward-prop
        output = model(data)
        # 評価しlossを計算
        loss = criterion(output, target)
        total_loss += loss.item()
        total_size += data.size(0)
        # back-prop
        loss.backward()
        # パラメータ更新
        optimizer.step()
        if batch_idx % 1000 == 0:
            now = datetime.datetime.now()
            print('[{}] Train Epoch: {}\tAverage loss: {:.6f}'.format(
                now,
                epoch,
                total_loss / total_size))

## 10.繰り返し学習させよう

一つの訓練データについて、何回も繰り返し学習を行います

In [10]:
for epoch in range(1, 3):
    train(epoch)

[2019-11-24 03:54:23.048070] Train Epoch: 1	Average loss: 0.216139
[2019-11-24 04:04:42.645970] Train Epoch: 1	Average loss: 0.072563


## 11. 評価モードにしよう

In [ ]:
model = model.eval()

## 12. 評価してみよう

In [12]:
pred = []
Y = []
for i, (x,y) in enumerate(test_loader):
    with torch.no_grad():
        # データをGPUに対応
        x = x.to(device)
        # モデルへ適応
        output = model(x)
    pred += [int(l.argmax()) for l in output]
    Y += [int(l) for l in y]

# 予測結果を評価する
print(classification_report(Y, pred))

              precision    recall  f1-score   support

           0       0.58      0.46      0.51      1000
           1       0.57      0.50      0.53      1000
           2       0.34      0.22      0.27      1000
           3       0.29      0.29      0.29      1000
           4       0.47      0.08      0.13      1000
           5       0.47      0.26      0.34      1000
           6       0.30      0.76      0.43      1000
           7       0.55      0.36      0.44      1000
           8       0.52      0.57      0.54      1000
           9       0.40      0.69      0.50      1000

    accuracy                           0.42     10000
   macro avg       0.45      0.42      0.40     10000
weighted avg       0.45      0.42      0.40     10000

